# Exploring Wide and Long

A notebook to explore wide and long data sets along with CSV for the web.   

Question:
* What is the point of wide vs long from the publisher POV?  
* Given modern workflows with data frame concepts across multiple languages and the popularity R DataFrames and Python Pandans, why not just empower the end user?   Dumb pipes, smart edges? 


References:
* [primer](https://www.w3.org/TR/tabular-data-primer/)
* [CSV-LD](https://www.w3.org/2013/csvw/wiki/CSV-LD) not sure what this group is
* [Python lib](https://github.com/cldf/csvw)


CSVW snippet for later
```
{
  "@context": {
   "@vocab": "http://www.w3.org/ns/csvw",
   "dc" : "http://whatever.dc.is#"
  },
  "url": "long.csv",
  "tableSchema": {
    "columns": [{
      "titles": "CaseID",
      "dc:description": "The internal caseID"
    },{
      "titles": "VariableRef"
    },{
      "titles": "Value"
    }]
  }
}
```

In [48]:
import pandas as pd

## Pangea example
https://doi.pangaea.de/10.1594/PANGAEA.855693

While the HTML view at https://doi.pangaea.de/10.1594/PANGAEA.855693?format=html#download knows how to skip comments this is likely either an observed convention (like CSV with comments) or code logic.  In pandas we can use comment or skiprows parameters to the call, but this has to be know.

* Can CSVW convey comment and skip row approaches for ingestion?
* Can CSWW denote deliminator?



In [49]:
distribution  = "https://doi.pangaea.de/10.1594/PANGAEA.855693?format=textfile"

png = pd.read_csv(distribution, index_col=False, sep='\t', comment='#', skiprows=31)

In [50]:
png.head(3)

,Date/Time,Latitude,Longitude,Depth water [m],Cast,Sample label,Chl a [µg/l],[NO3]- [µmol/l],Flag (NO3),[NO2]- [µmol/l],Flag (NO2),O2 [µmol/kg],Flag (Oxygen),[PO4]3- [µmol/l],Flag (PO4),Sal,SiO2 [µmol/l],Flag (SiO2),Temp [°C]
0,1957-04-30T00:00:00,54.5295,10.0393,1,1.0,1,NaN,NaN,NaN,NaN,NaN,321.9,NaN,0.00,NaN,15.3,NaN,NaN,7.7
1,1957-04-30T00:00:00,54.5295,10.0393,5,1.0,1,NaN,NaN,NaN,NaN,NaN,325.0,NaN,0.01,NaN,15.3,NaN,NaN,5.4
2,1957-04-30T00:00:00,54.5295,10.0393,10,1.0,1,NaN,NaN,NaN,NaN,NaN,325.0,NaN,0.02,NaN,15.7,NaN,NaN,6.1


## Info inspection

What follows is a simple inspection of the data frame with Pandas info.  We can see which columns map to inferred types.  Pandas dose a decent job only seeing Date/Time as something that is not identifiable as a data type.  

In [51]:
png.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5394 entries, 0 to 5393
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date/Time         5394 non-null   object 
 1   Latitude          5394 non-null   float64
 2   Longitude         5394 non-null   float64
 3   Depth water [m]   5394 non-null   int64  
 4   Cast              5388 non-null   float64
 5   Sample label      5394 non-null   int64  
 6   Chl a [µg/l]      3162 non-null   float64
 7   [NO3]- [µmol/l]   2569 non-null   float64
 8   Flag (NO3)        1091 non-null   float64
 9   [NO2]- [µmol/l]   2391 non-null   float64
 10  Flag (NO2)        1077 non-null   float64
 11  O2 [µmol/kg]      3961 non-null   float64
 12  Flag (Oxygen)     1090 non-null   float64
 13  [PO4]3- [µmol/l]  3254 non-null   float64
 14  Flag (PO4)        1092 non-null   float64
 15  Sal               5207 non-null   float64
 16  SiO2 [µmol/l]     2536 non-null   float64


## Inspect the first column

Let's naively guess the first column is our ID column. Can we do some sort of test to see if it implies long or wide data?  It obviously doesn't as it is a date time.  If we look through this the "sample label" might work.  It's not an "ID" but it is an label, let's see how many times it's used.

We really can't pivot this table since with 'Sample label' as a pivot index we will see that there is a high likelihood of a duplicate value.  As in there will be.

In [52]:
uid = png['Sample label'].unique()
print(len(uid))

473


Could we build an id that could be used as an IRI with some base URI prefix option?

In [53]:
# let's add an id column   (really now with melt)
# png['newid'] = range(1, len(png) + 1)
# png.head()

In [57]:
cols = png.columns
print(cols)  #  cheat here to get the labels easier  :)  

Index(['Date/Time', 'Latitude', 'Longitude', 'Depth water [m]', 'Cast',
       'Sample label', 'Chl a [µg/l]', '[NO3]- [µmol/l]', 'Flag (NO3)',
       '[NO2]- [µmol/l]', 'Flag (NO2)', 'O2 [µmol/kg]', 'Flag (Oxygen)',
       '[PO4]3- [µmol/l]', 'Flag (PO4)', 'Sal', 'SiO2 [µmol/l]', 'Flag (SiO2)',
       'Temp [°C]'],
      dtype='object')


In [82]:
# pd.wide_to_long(png, ["Flag"], i="newid", j="dt") #  oddly I could not get the wide to long to work but melt was easy

pdlg = pd.melt(png, id_vars=['Sample label'], value_vars=['Date/Time', 'Latitude', 'Longitude', 'Depth water [m]', 'Cast',
        'Chl a [µg/l]', '[NO3]- [µmol/l]', 'Flag (NO3)',
       '[NO2]- [µmol/l]', 'Flag (NO2)', 'O2 [µmol/kg]', 'Flag (Oxygen)',
       '[PO4]3- [µmol/l]', 'Flag (PO4)', 'Sal', 'SiO2 [µmol/l]', 'Flag (SiO2)',
       'Temp [°C]'])

pdlg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97092 entries, 0 to 97091
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sample label  97092 non-null  int64 
 1   variable      97092 non-null  object
 2   value         60756 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.2+ MB


In [78]:
pdlg.head() 

,Sample label,variable,value
0,1,Date/Time,1957-04-30T00:00:00
1,1,Date/Time,1957-04-30T00:00:00
2,1,Date/Time,1957-04-30T00:00:00
3,1,Date/Time,1957-04-30T00:00:00
4,1,Date/Time,1957-04-30T00:00:00


At this point it's easy to get the unique items in "variable".  Sample label can be cast to a unique URI via a base URI prefix and then using this.  Value then can be looked at as something that could be cast to a type.  

In [64]:
# let's make Arofan's unique ID
# date/time, latitude, longitude, and depth)

png['cmpdky'] =  png['Date/Time'].map(str) + '-' + png['Latitude'].map(str) + '-' + png['Longitude'].map(str) + '-' + png['Depth water [m]'].map(str)
png.head()

,Date/Time,Latitude,Longitude,Depth water [m],Cast,Sample label,Chl a [µg/l],[NO3]- [µmol/l],Flag (NO3),[NO2]- [µmol/l],Flag (NO2),O2 [µmol/kg],Flag (Oxygen),[PO4]3- [µmol/l],Flag (PO4),Sal,SiO2 [µmol/l],Flag (SiO2),Temp [°C],cmpdky
0,1957-04-30T00:00:00,54.5295,10.0393,1,1.0,1,NaN,NaN,NaN,NaN,NaN,321.9,NaN,0.00,NaN,15.3,NaN,NaN,7.7,1957-04-30T00:00:00-54.5295-10.0393-1
1,1957-04-30T00:00:00,54.5295,10.0393,5,1.0,1,NaN,NaN,NaN,NaN,NaN,325.0,NaN,0.01,NaN,15.3,NaN,NaN,5.4,1957-04-30T00:00:00-54.5295-10.0393-5
2,1957-04-30T00:00:00,54.5295,10.0393,10,1.0,1,NaN,NaN,NaN,NaN,NaN,325.0,NaN,0.02,NaN,15.7,NaN,NaN,6.1,1957-04-30T00:00:00-54.5295-10.0393-10
3,1957-04-30T00:00:00,54.5295,10.0393,15,1.0,1,NaN,NaN,NaN,NaN,NaN,318.8,NaN,0.03,NaN,16.4,NaN,NaN,4.5,1957-04-30T00:00:00-54.5295-10.0393-15
4,1957-04-30T00:00:00,54.5295,10.0393,20,1.0,1,NaN,NaN,NaN,NaN,NaN,300.0,NaN,0.06,NaN,17.0,NaN,NaN,4.3,1957-04-30T00:00:00-54.5295-10.0393-20


In [65]:
cmpdkyuid = png['cmpdky'].unique()
print(len(cmpdkyuid))

5284


In [66]:
print(len(png))

5394


The data frame has 5394 rows but only 5284 unique compound keys.  So we don't seem to have a unique ID for each row using that approach. 
However, we also don't have that with the previous "to long" melt data frame.  We can see that.

In [83]:
pdlg.groupby(pdlg.columns.tolist(),as_index=False).size()


,Sample label,variable,value,size
0,0,Cast,1,36
1,0,Chl a [µg/l],0.39,1
2,0,Chl a [µg/l],0.51,1
3,0,Chl a [µg/l],0.53,1
4,0,Chl a [µg/l],0.55,1
...,...,...,...,...
31307,487,[PO4]3- [µmol/l],0.97,1
31308,487,[PO4]3- [µmol/l],1.02,1
31309,487,[PO4]3- [µmol/l],1.03,1
31310,487,[PO4]3- [µmol/l],1.04,2


In [90]:
pdlgv2 = pdlg.drop_duplicates(keep = False)
pdlgv2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24215 entries, 2523 to 97084
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sample label  24215 non-null  int64 
 1   variable      24215 non-null  object
 2   value         24139 non-null  object
dtypes: int64(1), object(2)
memory usage: 756.7+ KB


In the above the 24139 non-null values show us that we DO have null object values which are rows we will want to remove.  

In [91]:
pdlgv2.head()

,Sample label,variable,value
2523,8,Date/Time,1996-03-23T00:00:00
3454,344,Date/Time,2003-03-11T09:42:00
4222,447,Date/Time,2011-07-25T10:02:25
4228,447,Date/Time,2011-07-25T10:19:52
16187,1,Depth water [m],26
